In [ ]:
"""in1_var tiene 13148 listas, cada una con 80 elementos que son listas que cada una tiene 120 elementos.
    Es un arreglo numpy de 3 dimensiones (3 ejes). El shape es el tamanio (cantidad de elementos) que tiene cada
    eje o dimension. El eje 0 o primer eje ES EL EJE "mas externo" (es lo que llamariamos la tercera dimension
    pero aca le dice eje 0), entonces en este caso el eje 0 es el que tiene las 13148 listas.
    
    expand_dims lo que hace es agregar un eje (una dimension) mas en el lugar que se le indique por el parametro
    axis. En este caso axis=3 entonces estaria creando una nueva dimension (nuestro axis=2 serian las listas que
    tenian 120 elementos cada una) o agregando un eje en la parte "MAS INTERNA" del arreglo:
    Entonces el arreglo expandido quedaria:
            13148 listas con 80 listas con 120 listas con 1 elementos cada una. (shape = 13148,80,120,1 y 4 ejes)
    
    Esto se hace con los 3 arreglos (z500, z700, z1000).
    
    concatenate toma una secuencia de arreglos y los une a traves del eje indicado.
    numpy.concatenate((a1, a2, ...#o sea una lista de arreglos a unir#), axis=#algunEje#)
    
    Aca se le pasa una lista de 3 arreglos cada uno con 4d (4 ejes y shape = 13148,80,120,1)
    y se le indica que los una a traves del eje=3, o sea del eje previamente agregado.
    
    Ej:
    arreglos con 2 listas, cada una con 3 listas de 2 elementos cada una (shape 2,3,2. 3 ejes)
    arr1= [[[ 1.  0.]
            [ 0.  0.]
            [ 0.  0.]]

           [[ 1.  0.]
            [ 0.  0.]
            [ 0.  0.]]]

    arr2= [[[ 2.  3.]
            [ 0.  0.]
            [ 0.  0.]]

           [[ 2.  3.]
            [ 0.  0.]
            [ 0.  0.]]]

    Si expando ambos arreglos agregando una dimension, quedaria:
    2 listas, cada una con 3 listas, cada una con 2 listas de 1 elemento cada una.
    shape = 2,3,2,1 . 4 ejes
    
  arr1= [[[[ 1.]
           [ 0.]]

          [[ 0.]
           [ 0.]]

          [[ 0.]
           [ 0.]]]


         [[[ 1.]
           [ 0.]]

          [[ 0.]
           [ 0.]]

          [[ 0.]
           [ 0.]]]]
           
  arr2= [[[[ 2.]
           [ 3.]]

          [[ 0.]
           [ 0.]]

          [[ 0.]
           [ 0.]]]


         [[[ 2.]
           [ 3.]]

          [[ 0.]
           [ 0.]]

          [[ 0.]
           [ 0.]]]]
    
    Y si concateno ambos arreglos a traves del nuevo eje (recordar que es el "mas interno"), los elementos se
    iran agregando y quedaria un arreglo de 2 listas con 3 listas con 2 listas CON 2 ELEMENTOS cada una.
    shape = 2,3,2,2 . 4 ejes"""

In [9]:
import numpy as np
import os.path
import sys
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.core import Flatten, Dense, Lambda
from keras import backend as K
from keras.optimizers import SGD, Adagrad, Adadelta
from keras.callbacks import CSVLogger

airports = ['EIDW', 'EGLL', 'LFPG', 'LFBO', 'EGPH', 'EHAM', 'EBBR', 'LEMD', 'LEBL', 'LPPT', 'LIRF',
            'LIMC', 'LSZH', 'EDDM', 'EDFH', 'EDDT', 'EKCH', 'ENGM', 'ESSA', 'EFHK', 'LOWW']

def get_rains(code):
    arr = np.load("/home/guilletrejo/DeepWeather-master/short/rain_5000.npy")
    idx = airports.index(code)
    return arr[:, idx].astype(np.int32)

def get_era_full(param, level):
    arr = np.load("/home/guilletrejo/DeepWeather-master/short/{}{}_5000_uint8.npy".format(param, level))
    return arr.astype(np.float32) / 256.

def train_model(airport):
    model = Sequential()
    model.add(Convolution2D(128, (3, 3), padding='same', activation='relu', name='block1_conv1', input_shape=(80,120,3)))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Convolution2D(256, (3, 3), padding='same', activation='relu', name='block2_conv1'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Flatten())
    model.add(Dense(2, activation = 'softmax', name='final_fully_connected'))

    # Import data
    params = ["z", "z", "z"]
    levels = [500, 700, 1000]

    in1_var = get_era_full(params[0], levels[0])
    in2_var = get_era_full(params[1], levels[1])
    in3_var = get_era_full(params[2], levels[2])

    print("Shape in1 = " + str(in1_var.shape))
    print("Shape in2 = " + str(in2_var.shape))
    print("Shape in3 = " + str(in3_var.shape))

    # Agrega un eje mas interno a los arreglos input
    in1_var_ex = np.expand_dims(in1_var, axis=3)
    in2_var_ex = np.expand_dims(in2_var, axis=3)
    in3_var_ex = np.expand_dims(in3_var, axis=3)

    print("Shape in1_ex = " + str(in1_var_ex.shape))
    print("Shape in2_ex = " + str(in2_var_ex.shape))
    print("Shape in3_ex = " + str(in3_var_ex.shape))
    
    # Concatena los 3 input de 4 dimensiones y shape 140,80,120,1 en un solo input 
    # de 4 dimensiones con shape 140,80,120,3
    X = np.concatenate((in1_var_ex, in2_var_ex, in3_var_ex), axis=3)
    print(X.shape)
    
    Y = get_rains(airport)
    b = np.zeros((Y.shape[0], 2))
    b[np.arange(Y.shape[0]), Y] = 1

    print("Shape Y = " + str(Y.shape))
    print("Shape b = " + str(b.shape))
    
    adagrad = Adagrad(lr=0.0002)
    model.compile(loss='categorical_crossentropy', optimizer = adagrad, metrics=['accuracy'])

    csv_logger = CSVLogger('{}.log'.format(airport))
    model.fit(X, b, batch_size=100, epochs=100, verbose=1, validation_split=0.2, callbacks=[csv_logger])

    model.save('model_2d_{}.h5'.format(airport))
    return model



In [10]:
if __name__ == '__main__':
    airport = 'EIDW'
    model1 = train_model(airport)

Shape in1 = (5000, 80, 120)
Shape in2 = (5000, 80, 120)
Shape in3 = (5000, 80, 120)
Shape in1_ex = (5000, 80, 120, 1)
Shape in2_ex = (5000, 80, 120, 1)
Shape in3_ex = (5000, 80, 120, 1)
(5000, 80, 120, 3)
Shape Y = (5000,)
Shape b = (5000, 2)
Train on 4000 samples, validate on 1000 samples
Epoch 1/100
4000/4000 [==============================] - 391s 98ms/step - loss: 0.6865 - acc: 0.5902 - val_loss: 0.6510 - val_acc: 0.6350 - ETA: 13:00 - loss: 0.9182 - acc: 0.6150

In [11]:
X1 = np.load("/home/guilletrejo/DeepWeather-master/short/z500_shorter_uint8.npy")
X2 = np.load("/home/guilletrejo/DeepWeather-master/short/z700_shorter_uint8.npy")
X3 = np.load("/home/guilletrejo/DeepWeather-master/short/z1000_shorter_uint8.npy")

In [12]:
X1_ex = np.expand_dims(X1,axis=3)
X2_ex = np.expand_dims(X2,axis=3)
X3_ex = np.expand_dims(X3,axis=3)
Xall = np.concatenate((X1_ex,X2_ex,X3_ex),axis=3)
print(Xall.shape)

(100, 80, 120, 3)


In [13]:
def get_rains(code):
    arr = np.load("/home/guilletrejo/DeepWeather-master/short/rain_shorter.npy")
    idx = airports.index(code)
    return arr[:, idx].astype(np.int32)

y = get_rains('EIDW')
print(y.shape)
b = np.zeros((y.shape[0], 2))
b[np.arange(y.shape[0]), y] = 1
print(y.shape)
print(b.shape)


print(model1.evaluate(Xall,b))
prediction = model1.predict(Xall,verbose=1)

(100,)
(100,)
(100, 2)
100/100 [==============================] - 3s 27ms/step
[5.9636954498291015, 0.63]
100/100 [==============================] - 3s 32ms/step


In [30]:
#print(np.concatenate((prediction,b), axis=1).astype(np.int32))

count = 0

for i in range(len(prediction)):
    if (prediction[i]==b[i]).all():
        count += 1

print(count)
print(len(prediction))
print(len(b))

58
100
100


In [62]:
#model1.save('model_2d_1epoch_shorter_train_EIDW.h5')
#model1.save('model_2d_1epoch_shorter_train_EIDW.h5')